## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,description
0,0,Hilo,19.7297,-155.0900,72.25,89,46,1.01,US,2022-02-14 04:04:46,scattered clouds
1,1,Ponta Do Sol,32.6667,-17.1000,61.05,74,94,1.77,PT,2022-02-14 04:01:38,overcast clouds
2,2,Ushuaia,-54.8000,-68.3000,33.10,92,31,4.85,AR,2022-02-14 04:05:05,scattered clouds
3,3,Luderitz,-26.6481,15.1594,70.43,71,0,8.90,NaN,2022-02-14 04:06:06,clear sky
4,4,Tuktoyaktuk,69.4541,-133.0374,-34.35,100,71,7.61,CA,2022-02-14 04:06:07,broken clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,description
7,7,Bambous Virieux,-20.3428,57.7575,82.22,86,100,13.47,MU,2022-02-14 04:06:08,light rain
19,19,Hobyo,5.3505,48.5268,76.91,68,50,16.60,SO,2022-02-14 04:06:13,scattered clouds
20,20,Isangel,-19.5500,169.2667,84.63,72,64,12.30,VU,2022-02-14 04:06:14,broken clouds
30,30,Atuona,-9.8000,-139.0333,77.31,80,22,13.73,PF,2022-02-14 04:06:17,light rain
34,34,Kawalu,-7.3817,108.2082,84.00,72,86,1.86,ID,2022-02-14 04:06:19,overcast clouds
35,35,Antalaha,-14.9003,50.2788,75.00,88,64,3.65,MG,2022-02-14 04:06:20,broken clouds
38,38,Bathsheba,13.2167,-59.5167,77.61,73,40,16.11,BB,2022-02-14 04:06:20,scattered clouds
43,43,Rikitea,-23.1203,-134.9692,78.91,71,2,14.07,PF,2022-02-14 04:06:22,clear sky
47,47,Vaini,-21.2000,-175.2000,80.13,71,91,16.04,TO,2022-02-14 04:07:24,overcast clouds
61,61,Busselton,-33.6500,115.3333,89.04,40,0,5.10,AU,2022-02-14 04:04:07,clear sky


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        0
Date           0
description    0
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,description,Lat,Lng,Hotel Name
7,Bambous Virieux,MU,82.22,light rain,-20.3428,57.7575,
19,Hobyo,SO,76.91,scattered clouds,5.3505,48.5268,
20,Isangel,VU,84.63,broken clouds,-19.5500,169.2667,
30,Atuona,PF,77.31,light rain,-9.8000,-139.0333,
34,Kawalu,ID,84.00,overcast clouds,-7.3817,108.2082,
35,Antalaha,MG,75.00,broken clouds,-14.9003,50.2788,
38,Bathsheba,BB,77.61,scattered clouds,13.2167,-59.5167,
43,Rikitea,PF,78.91,clear sky,-23.1203,-134.9692,
47,Vaini,TO,80.13,overcast clouds,-21.2000,-175.2000,
61,Busselton,AU,89.04,clear sky,-33.6500,115.3333,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()
clean_hotel_df = hotel_df.dropna()

In [34]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))